<a href="https://colab.research.google.com/github/litkm/WJBrennan-Voting/blob/main/WJ_Brennen_1980s_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#Inspiration for this project came from lesson 8 of Prof. Wolfgang Alschner's course, Data Science for Lawyers (https://www.datascienceforlawyers.org/learning-resources/lesson-8/)
#Model adapted from the classification project in unit 4 of Codecademy's skills path, Build Deep Learning Models with TensorFlow

import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import numpy as np
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization

dataset = 'https://raw.githubusercontent.com/litkm/WJBrennan-Voting/main/WJBrennan_voting.csv'
dataset = pd.read_csv(dataset)

#print(dataset.head())
#print(dataset.describe())
print(dataset.info())

print('Classes and number of values in the dataset',Counter(dataset['vote']))

y = dataset['vote']
x = dataset[['term', 'petitioner', 'respondent', 'jurisdiction', 'caseOrigin', 'caseSource', 'certReason', 'issue', 'issueArea']]

#y = pd.get_dummies(y) #is this necessary or do we process it later?

#X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

Y_train = y.iloc[0:3368]      #Labels for training
Y_test = y.iloc[3367:4745]    #Labels for testing

X_train = x[x['term'] < 1980] #Features for training
X_test = x[x['term'] > 1979]  #Features for testing

columns_for_standard = ['term', 'petitioner', 'respondent', 'jurisdiction', 'caseOrigin', 'caseSource', 'certReason', 'issue', 'issueArea']

ct = ColumnTransformer([('numeric', StandardScaler(), columns_for_standard)])

X_train = ct.fit_transform(X_train) 
X_test = ct.transform(X_test)

le = LabelEncoder()
Y_train = le.fit_transform(Y_train.astype(str))
Y_test = le.transform(Y_test.astype(str))

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],)))
model.add(BatchNormalization())

model.add(Dense(18, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=100, batch_size=8, verbose=1, validation_data=(X_test, Y_test))

loss, acc = model.evaluate(X_test, Y_test, verbose=0)
print("Loss:", loss, "Accuracy:", acc)

y_estimate = model.predict(X_test, verbose=0)
y_estimate = np.argmax(y_estimate, axis=1)
y_true = np.argmax(Y_test, axis=1)
print(classification_report(y_true, y_estimate))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   term          4746 non-null   int64 
 1   petitioner    4746 non-null   int64 
 2   respondent    4746 non-null   int64 
 3   jurisdiction  4746 non-null   int64 
 4   caseOrigin    4746 non-null   int64 
 5   caseSource    4746 non-null   int64 
 6   certReason    4746 non-null   int64 
 7   issue         4746 non-null   int64 
 8   issueArea     4746 non-null   int64 
 9   vote          4746 non-null   object
dtypes: int64(9), object(1)
memory usage: 370.9+ KB
None
Classes and number of values in the dataset Counter({'majority': 3683, 'dissent': 1063})
Epoch 1/100
421/421 [==============================] - 2s 2ms/step - loss: 0.8006 - accuracy: 0.6210 - val_loss: 0.7314 - val_accuracy: 0.5479
Epoch 2/100
421/421 [==============================] - 1s 2ms/step - loss: 0.5070 - accuracy: 0.7886